In [1]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 2.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=62f3cac04a18bf69c35c50e3d7af87bed56784276419d9a67592280ea59845c1
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [2]:
!pip install Transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 18.4 MB/s 
     |████████████████████████████████| 163 kB 51.0 MB/s 
     |████████████████████████████████| 7.6 MB 63.7 MB/s 


In [3]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [6]:
import sys
sys.path.insert(0,'/content/drive/My Drive/tcc-colab/')

In [7]:
import logging
import numpy as np
from seqeval.metrics import accuracy_score, precision_score
from torch import nn
from transformers import DistilBertTokenizerFast, TrainingArguments, Trainer, \
    BertForTokenClassification, EvalPrediction, AutoConfig
from seqeval.metrics import recall_score, f1_score
from ner.bert.fine_tuning import __processar_base

In [8]:
unique_tags, train_dataset, val_dataset, tokenizer, id2tag, tag2id = __processar_base()

Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizerFast'.


FileNotFoundError: ignored

In [ ]:
tag2id

In [ ]:
configuracao = AutoConfig.from_pretrained("neuralmind/bert-large-portuguese-cased", id2label=id2tag,
                                              label2id=tag2id)

In [ ]:
model = BertForTokenClassification.from_pretrained("neuralmind/bert-large-portuguese-cased",
                                                       config=configuracao)

In [ ]:
training_args = TrainingArguments(
        output_dir='./results4',  # output directory
        num_train_epochs=3,  # total number of training epochs
        # per_device_train_batch_size=16,  # batch size per device during training
        # per_device_eval_batch_size=64,  # batch size for evaluation
        per_device_train_batch_size=1,  # batch size per device during training
        per_device_eval_batch_size=1,  # batch size for evaluation
        warmup_steps=500,  # number of warmup steps for learning rate scheduler
        weight_decay=0.01,  # strength of weight decay
        logging_steps=10
    )

In [ ]:
from collections import Counter
from matplotlib import pyplot as plt

label_list = list(tag2id)
label_list

In [ ]:

dsets = [train_dataset,val_dataset]
num=0
for dset in dsets:
    ner_tags_list = list()
    for i,row in enumerate(dset.labels):
        sublist= list()
        for r in row:
            if r == -100:
                continue
            sublist.append(r)
        ner_tags_list += sublist
    ner_tags_dist = Counter(ner_tags_list)
    ner_tags_dist = {label_list[k]:v for k,v in ner_tags_dist.items()}
    sorted_ner_tags_dist = {k:ner_tags_dist[k] for k in label_list}
    ner_tags_dist = Counter(ner_tags_list)
    ner_tags_dist = {label_list[k]:v for k,v in ner_tags_dist.items()}
    sorted_ner_tags_dist = {k:ner_tags_dist[k] for k in label_list}

    fig = plt.figure(figsize=(15,5))
    axes = fig.subplots(nrows=1, ncols=2)
    name= "Dados de Treinamento" if not num else "Dados de Validação"
    for i,col in enumerate(axes):
            if i == 0:
                col.bar(sorted_ner_tags_dist.keys(), sorted_ner_tags_dist.values(), width=1, color='g')
                col.set_xticklabels(labels=list(sorted_ner_tags_dist.keys()),rotation=45)

                col.set_title(f'{name} - Todas Entidades')
            if i == 1:
                sorted_ner_tags_dist_without_O = sorted_ner_tags_dist.copy()
                del_value = sorted_ner_tags_dist_without_O.pop('O')
                col.bar(sorted_ner_tags_dist_without_O.keys(), sorted_ner_tags_dist_without_O.values(), width=1, color='g')
                col.set_xticklabels(labels=list(sorted_ner_tags_dist_without_O.keys()),rotation=45)
                col.set_title(f'{name} - Sem a entidade 0 (não classificada)')
    num+=1
    plt.show()

In [ ]:
def align_predictions(predictions: np.ndarray, label_ids: np.ndarray):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
                out_label_list[i].append(id2tag[label_ids[i][j]])
                preds_list[i].append(id2tag[preds[i][j]])

    return preds_list, out_label_list

In [ ]:
def compute_metrics(p: EvalPrediction) -> dict:
    preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

In [ ]:
trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=val_dataset,  # evaluation dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
eval = trainer.evaluate(val_dataset)

In [ ]:
eval

In [ ]:
trainer.save_model("/content/drive/My Drive/tcc-colab/results5")

In [ ]:
!zip -r /content/results4.zip /content/results4/

In [ ]:
from google.colab import files
files.download('results4.zip')